In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
#Lettura dataframe
play_store = pd.read_csv("https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore.csv")
play_store_reviews = pd.read_csv("https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore_user_reviews.csv")

#Rimozione riga errata
play_store = play_store.drop(play_store[play_store.Category == '1.9'].index)

In [3]:
#numero di righe del dataframe
rows_number = len(play_store['App'])

# 1. Convert the app sizes to a number

In [4]:
#regex per xxM e xxk
m = re.compile('[0-9]M')
k = re.compile('[0-9]k')

#lista 'Size'
sizes = list(play_store['Size'])

#modifica 'Size'
for i in range(0, rows_number):
    old = sizes[i]
    if(re.search(m, old) != None): #abbiamo trovato il pattern 'xxM'
        sizes[i] = float(old.split('M')[0])*1000000
    else:
        if(re.search(k, old) != None):
            sizes[i] = float(old.split('k')[0])*1000

#modifica della colonna 'Size' del dataframe originale con i nuovi valori
play_store['Size'] = sizes

# 2. Convert the number of installs to a number

In [5]:
#Lista Installs
inst = list(play_store['Installs'])

In [6]:
#Modifica Installs
for i in range(0, rows_number):
    x = (inst[i].split('+')[0])
    inst[i] = x.replace(',', '')

#Conversione in int
for i in range(0, rows_number):
    inst[i] = int(inst[i])

#Modifica della colonna 'Size' del dataframe originale con i nuovi valori
play_store["Installs"] = inst

# 3. Transform “Varies with device” into a missing value

In [7]:
play_store = play_store.replace('Varies with device', np.nan)

# 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [8]:
#Liste Current Ver e Android Ver
cur_ver = list(play_store['Current Ver'])
andr_ver = list(play_store['Android Ver'])

#Conversione
for i in range(0, rows_number):
    if(re.search(re.compile('[0-9][.][0-9]([.][0-9])*'), str(cur_ver[i])) != None):
        cur_ver[i] = cur_ver[i].replace(cur_ver[i], re.search(re.compile('[0-9][.][0-9]([.][0-9])*'), cur_ver[i]).group(0))
        
for i in range(0, rows_number):
    if(re.search(re.compile('[0-9][.][0-9]([.][0-9])*'), str(andr_ver[i])) != None):
        andr_ver[i] = andr_ver[i].replace(andr_ver[i], re.search(re.compile('[0-9][.][0-9]([.][0-9])*'), andr_ver[i]).group(0))

#Modifica delle colonne 'Current Ver' e 'Android Ver' del dataframe originale con i nuovi valori
play_store['Current Ver'] = cur_ver
play_store['Android Ver'] = andr_ver

# 5. Remove the duplicates

In [9]:
#numero di righe identiche
play_store.duplicated().sum()

#eliminazione duplicati righe identiche
play_store = play_store.drop_duplicates(subset=None, keep='first', inplace=False)

In [10]:
#eliminazione righe con campi 'App' identici (per ogni applicazione presente più di una volta, mantengo la riga che ha il maggior numero di recensioni)
play_store = play_store.sort_values(by = ['App', 'Reviews'], ascending=False).drop_duplicates('App', keep = 'first', inplace = False).sort_index().reset_index(drop=True)

In [11]:
rows_number = len(play_store['App'])

# 6. For each category, compute the number of apps

In [12]:
play_store[["App"]].groupby(play_store["Category"]).count()

,App
Category,
ART_AND_DESIGN,61
AUTO_AND_VEHICLES,85
BEAUTY,53
BOOKS_AND_REFERENCE,222
BUSINESS,420
COMICS,56
COMMUNICATION,315
DATING,171
EDUCATION,108


# 7. For each category, compute the average rating

In [13]:
play_store[["Rating"]].groupby(play_store["Category"]).mean()

,Rating
Category,
ART_AND_DESIGN,4.359322
AUTO_AND_VEHICLES,4.190411
BEAUTY,4.278571
BOOKS_AND_REFERENCE,4.344970
BUSINESS,4.098479
COMICS,4.181481
COMMUNICATION,4.121484
DATING,3.970149
EDUCATION,4.358879


# 8. Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [14]:
#Creazione df dei generi
genres = list(set(play_store['Genres']))
genres_len = len(genres)

for i in range(0, genres_len):
    if(re.search(re.compile('[;]'), genres[i])):
        x = genres[i]
        genres[i] = x.split(';')[0]
        genres.append(x.split(';')[1])
        
genres = list(set(genres))
genres_len = len(genres)
genres_df = pd.DataFrame(genres, columns = ['Genre'])


#creazione df app e generi
apps_genres_df = pd.DataFrame(columns=["App", "Genre"])
for i in play_store.values:
    for j in i[9].split(";"): 
        x = pd.Series([i[0], j], index = apps_genres_df.columns)
        apps_genres_df = apps_genres_df.append(x, ignore_index = True)


# 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [15]:
#creo un dataframe con i generi come colonne
bool_genres = pd.DataFrame()
for i in range(0, genres_len):
    x = pd.DataFrame(data = np.full(shape = len(play_store['Genres']), fill_value = False, dtype = bool), columns = [str(genres[i])])
    bool_genres = pd.concat([bool_genres, x], axis = 1)

#join tra il dataframe di riferimento e quello con i generi appena creato
play_store = play_store.join(bool_genres, sort = False)


In [16]:
pd.options.mode.chained_assignment = None  # default='warn'

#Assegno True
for i in range(0, rows_number):
    if(re.search(re.compile('[;]'), (play_store['Genres'][i]))):
        play_store[str(play_store['Genres'][i].split(';')[0])][i] = True
        play_store[str(play_store['Genres'][i].split(';')[1])][i] = True
    else:
        play_store[str(play_store['Genres'][i])][i] = True    

# 10. For each genre, compute the average rating. What is the genre with highest average?

In [28]:
apps_genres_df.join(play_store["Rating"], sort = False)[["Rating"]].groupby(apps_genres_df["Genre"]).mean().sort_values(by = 'Rating', ascending = False)

,Rating
Genre,
Creativity,4.404000
Auto & Vehicles,4.315278
Parenting,4.312069
Travel & Local,4.297207
Art & Design,4.280328
Beauty,4.276087
Health & Fitness,4.258824
Shopping,4.245506
Food & Drink,4.242593


# 11. For each app, compute the approximate income, obtain as a product of number of installs and price

In [18]:
play_store["Income"] = pd.Series()
for i in range(0, rows_number):
    play_store["Income"][i] = play_store["Installs"][i] * float(play_store["Price"][i].replace('$', ''))

# 12. For each app, compute its minimum and maximum Sentiment_polarity

In [19]:
#min
play_store_reviews[["Sentiment_Polarity"]].groupby(play_store_reviews['App']).min()

,Sentiment_Polarity
App,
10 Best Foods for You,-0.800000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.112500
11st,-1.000000
1800 Contacts - Lens Store,-0.300000
1LINE – One Line with One Touch,-0.825000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,-0.800000
21-Day Meditation Experience,-0.265625
"2Date Dating App, Love and matching",-0.645833
2GIS: directory & navigator,-0.375000


In [20]:
#max
play_store_reviews[["Sentiment_Polarity"]].groupby(play_store_reviews['App']).max()

,Sentiment_Polarity
App,
10 Best Foods for You,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000
11st,1.000000
1800 Contacts - Lens Store,0.838542
1LINE – One Line with One Touch,1.000000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,1.000000
21-Day Meditation Experience,0.587500
"2Date Dating App, Love and matching",1.000000
2GIS: directory & navigator,1.000000


In [23]:
play_store

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Social,Books & Reference,Arcade,Parenting,Events,Adventure,Medical,Auto & Vehicles,Simulation,Income
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0
2,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000.0,50000000,Free,0,Teen,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0
3,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,0.0
4,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5600000.0,50000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0
5,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19000000.0,50000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0
6,Infinite Painter,ART_AND_DESIGN,4.1,36815,29000000.0,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0
7,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33000000.0,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0
8,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3100000.0,10000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,0.0
9,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,13880,28000000.0,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,0.0


In [21]:
genres_df

,Genre
0,Card
1,Music & Audio
2,Action & Adventure
3,Racing
4,Weather
5,Travel & Local
6,Art & Design
7,Word
8,Trivia
9,Role Playing


In [22]:
apps_genres_df

,App,Genre
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design
2,Sketch - Draw & Paint,Art & Design
3,Pixel Draw - Number Art Coloring Book,Art & Design
4,Pixel Draw - Number Art Coloring Book,Creativity
5,Paper flowers instructions,Art & Design
6,Smoke Effect Photo Maker - Smoke Editor,Art & Design
7,Infinite Painter,Art & Design
8,Garden Coloring Book,Art & Design
9,Kids Paint Free - Drawing Fun,Art & Design
